## Example 1

In [1]:
def dy_dx(x):
    return 2 * x

In [2]:
import torch
dy_dx(3)

6

In [3]:
x = torch.tensor(3.0, requires_grad=True)
y = x ** 2

In [4]:
print(x)
print(y)

tensor(3., requires_grad=True)
tensor(9., grad_fn=<PowBackward0>)


In [5]:
y.backward()

In [6]:
x.grad

tensor(6.)

## Example 2

In [7]:
import math

In [8]:
def dz_dx(x):
    return 2 * x * math.cos(x ** 2)

In [9]:
dz_dx(3)

-5.466781571308061

In [10]:
x = torch.tensor(3.0, requires_grad=True)
y = x ** 2
z = torch.sin(y)

In [11]:
print(x,y,z)

tensor(3., requires_grad=True) tensor(9., grad_fn=<PowBackward0>) tensor(0.4121, grad_fn=<SinBackward0>)


In [12]:
z.backward()

In [13]:
x.grad

tensor(-5.4668)

## Example 3

#### Using manual derivative calculation

In [25]:
y = 0
w = 1
b = 0
z = w * x + b
y_pred = torch.sigmoid(z) # Used torch function for ease, should use manual calculation here

# 1. dL/d(y_pred): Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

# 2. dy_pred/dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x  # dz/dw = x
dz_db = 1  # dz/db = 1 (bias contributes directly to z)

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [26]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db}")

Manual Gradient of loss w.r.t weight (dw): 6.691762447357178
Manual Gradient of loss w.r.t bias (db): 0.998770534992218


#### Using pytorch

In [27]:
# Binary Cross-Entropy Loss for scalar
def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8  # To prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [50]:
# Inputs
x = torch.tensor(6.7,requires_grad=True)  # Input feature
y = torch.tensor(0.0)  # True label (binary)

w = torch.tensor(1.0, requires_grad=True)  # Weight
b = torch.tensor(1.0)  # Bias

In [51]:

# Forward pass
z = w * x + b  # Weighted sum (linear part)
y_pred = torch.sigmoid(z)  # Predicted probability

# Compute binary cross-entropy loss
loss = binary_cross_entropy_loss(y_pred, y)

In [52]:
loss.backward()

In [53]:
x.grad,w.grad

(tensor(0.9995), tensor(6.6970))

### For a vector input

In [61]:
x = torch.tensor([1.0,2.0,3.0],requires_grad=True)


In [62]:
y = (x ** 2).mean()

In [63]:
y.backward()

In [64]:
x.grad # Returns 3 gradients, one for each element in x

tensor([0.6667, 1.3333, 2.0000])

#### Clearing gradients

Pytorch by default accumulates gradients. These gradients need to be reset with every batch of training data.

In [65]:
x.grad.zero_() # Reset gradients

tensor([0., 0., 0.])

In [66]:
x = torch.tensor(3.0, requires_grad=True)

In [67]:
z = x.detach() # Detach from the graph